In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS increamental_load.default.raw_upi_transaction_v1
(
  transaction_id STRING,
  upi_id STRING,
  merchent_id STRING,
  transaction_amount double,
  transaction_timestamp TIMESTAMP,
  transaction_status STRING
)
USING DELTA
TBLPROPERTIES (delta.enableChangeDataCapture = true)
""")

# dbutils.fs.ls("/increamental_load/default/")

print("Delta table increamental_load.default.raw_upi_transaction_v1")


In [0]:
from delta.tables import DeltaTable
import time 

mock_batches = [
    # Batch 1: Insert new transactions
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0,  "2024-12-21 10:00:00", "initiated"),
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:05:00", "initiated"),
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:10:00", "initiated"),
    ], ['transaction_id', 'upi_id', 'merchent_id', 'transaction_amount', 'transaction_timestamp', 'transaction_status']),

    # Batch 2: Update and insert transactions
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:15:00", "completed"),  # Update transaction
        ("T002", "upi2@bank", "M002", 1000.0, "2024-12-21 10:20:00", "failed"),    # Update transaction
        ("T004", "upi4@bank", "M004", 2000.0, "2024-12-21 10:25:00", "initiated"), # New transaction
    ], ['transaction_id', 'upi_id', 'merchent_id', 'transaction_amount', 'transaction_timestamp', 'transaction_status']),

    # Batch 3: Handle refunds and updates
    spark.createDataFrame([
        ("T001", "upi1@bank", "M001", 500.0, "2024-12-21 10:30:00", "refunded"),  # Refund issued
        ("T003", "upi3@bank", "M003", 1500.0, "2024-12-21 10:35:00", "completed"), # Completed transaction
    ], ['transaction_id', 'upi_id', 'merchent_id', 'transaction_amount', 'transaction_timestamp', 'transaction_status']),
]




In [0]:
def merge_delta_table(delta_table_name:str, batch_df):
    print(f"Table name: {delta_table_name} ")
    delta_table = DeltaTable.forName(spark, delta_table_name)

    merge_condition="target.transaction_id=source.transaction_id"
    delta_table.alias("target").merge(
        batch_df.alias('source'),merge_condition)\
            .whenMatchedUpdate(
                set={
                "upi_id":"source.upi_id",
                "merchent_id":"source.merchent_id",
                "transaction_amount":"source.transaction_amount",
                "transaction_timestamp":"source.transaction_timestamp",
                "transaction_status":"source.transaction_status"
            }).whenNotMatchedInsertAll()\
                .execute()
    
 
merge_delta_table(f"increamental_load.default.raw_upi_transaction_v1",mock_batches[2])



In [0]:
mock_batches[2].show()

In [0]:
%sql
use catalog `increamental_load`; select * from `default`.`aggregated_upi_transactions` limit 100;